# Automatización Programación a Base de Datos

## Importamos Librerias

In [137]:
import pandas as pd
import numpy as np
import calendar
from datetime import timedelta

## Definimos funciones para lectura de Excel

In [138]:
def int_to_excel_col(n):
    result = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        result = chr(65 + remainder) + result
    return result

def excel_col_to_int(col):
    result = 0
    for char in col:
        result = result * 26 + (ord(char.upper()) - ord('A') + 1)
    return result

def next_excel_col(col):
    col_int = excel_col_to_int(col)
    next_col_int = col_int + 1
    return int_to_excel_col(next_col_int)

## Parte 1:

Completar campos de CC, Nombre BT, Producto, Puerto, Volumen, ETA y Fin descarga.

In [139]:
PATH_PROGRAMACION = "Programacion Descarga Importaciones 22 de Sep.xlsx"
PATH_DISTANCIAS = "Distancias entre puertos.xlsx"

### Paso 1:

Extraemos todos los programas, con sus respectivos buques y abreviaturas

- Problema: Abreviaturas duplicadas en hoja de buques (Seaways Kenosha-Seaways Kolberg).

In [140]:
def extraer_bts(file, sheet):
    df_bts = pd.read_excel(file, sheet_name=sheet, header=0)
    df_bts.index = range(1, len(df_bts) + 1)
    df_bts = df_bts[["N° Referencia", "Nombre programa", "Nombre del BT", "Abrev."]]
    df_bts.drop_duplicates(subset=["N° Referencia"], keep="first", inplace=True)
    
    return df_bts

In [141]:
df_bts = extraer_bts(PATH_PROGRAMACION, "Buques")
df_bts.index = range(1, len(df_bts) + 1)
df_bts

,N° Referencia,Nombre programa,Nombre del BT,Abrev.
1,CC 135/25,135/25 Seaways Kenosha,Seaways Kenosha,SKE
2,CC 117/25,117/25 Yaca,Yaca,Yac
3,CC 46/25,46/25 Sakura Voyager,Sakura Voyager,SV
4,CC 19/25,19/25 Cururo,Cururo,Cur
5,CC 47/25,47/25 San Jack,San Jack,SJ
6,CC 102/25,102/25 Pintail Pacific,Pintail Pacific,PP
7,CC 75/25,75/25 Flora Express,Flora Express,FE
8,CC 20/25,20/25 Seaways Kolberg,Seaways Kolberg,SK
9,CC 48/25,48/25 STI Mighty,STI Mighty,SM
10,CC 21/25,21/25 Seaways Athens,Seaways Athens,SA


### Paso 2: 

Creamos un dataframe con los productos y las plantas presentes en la planificación.

* Se asocia la ciudad de calbuco a la planta pureo.
* Alias según formato utilizado en la base de datos de estimación semanal.

In [142]:
df_productos = pd.DataFrame({"Producto": ["Diesel A1", "Gas 93", "Gas 97", "Jet A1"]})
df_plantas = pd.DataFrame({"Planta": ["PLANTA IQUIQUE", "PLANTA MEJILLONES", "PLANTA CALDERA",
                                      "TERMINAL TPI", "OXIQUIM QUINTERO", "OXIQUIM CORONEL",
                                      "PLANTA PUREO", "ENAP QUINTERO", "BT PUMA"],
                           "Ciudad": ["Iquique", "Mejillones", "Caldera", "Quintero", "Quintero",
                                      "Coronel", "Calbuco", np.nan, np.nan],
                           "Alias": ["Iquique", "Mejillones", "Caldera", "TPI", "Oxiquim Quintero",
                                      "Coronel", "Pureo", "Quintero", "Puma"]})
df_plantas.index = range(1, len(df_plantas) + 1)
df_plantas

,Planta,Ciudad,Alias
1,PLANTA IQUIQUE,Iquique,Iquique
2,PLANTA MEJILLONES,Mejillones,Mejillones
3,PLANTA CALDERA,Caldera,Caldera
4,TERMINAL TPI,Quintero,TPI
5,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero
6,OXIQUIM CORONEL,Coronel,Coronel
7,PLANTA PUREO,Calbuco,Pureo
8,ENAP QUINTERO,NaN,Quintero
9,BT PUMA,NaN,Puma


### Paso 3: 

Producto cruz entre plantas y productos para generar todas las combinaciones posibles.

In [143]:
df_productos_plantas = df_productos.merge(df_plantas, how='cross')
df_productos_plantas.index = range(1, len(df_productos_plantas) + 1)
df_productos_plantas

,Producto,Planta,Ciudad,Alias
1,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
2,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones
3,Diesel A1,PLANTA CALDERA,Caldera,Caldera
4,Diesel A1,TERMINAL TPI,Quintero,TPI
5,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero
6,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel
7,Diesel A1,PLANTA PUREO,Calbuco,Pureo
8,Diesel A1,ENAP QUINTERO,NaN,Quintero
9,Diesel A1,BT PUMA,NaN,Puma
10,Gas 93,PLANTA IQUIQUE,Iquique,Iquique


### Paso 4: 

Asignamos a cada par producto-planta la columna correspondiente en la planificación.

In [144]:
df_productos_plantas["Columna"] = ["M", "S", "Y", "AE", "BC", "BI", "BO", "BU", "CA",
                                   np.nan, "DA", np.nan, "AQ", np.nan, np.nan, np.nan, np.nan, "CK",
                                   np.nan, np.nan, np.nan, np.nan, "AW", np.nan, np.nan, np.nan, "CP",
                                   np.nan, "CU", np.nan, "AK", np.nan, np.nan, np.nan, np.nan, "CF"]

### Paso 5: 

Limpiamos el dataframe eliminando las filas sin columna asignada.

In [145]:
df_productos_plantas.dropna(subset=["Columna"], inplace=True)
df_productos_plantas.index = range(1, len(df_productos_plantas) + 1)
df_productos_plantas

,Producto,Planta,Ciudad,Alias,Columna
1,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,M
2,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,S
3,Diesel A1,PLANTA CALDERA,Caldera,Caldera,Y
4,Diesel A1,TERMINAL TPI,Quintero,TPI,AE
5,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,BC
6,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,BI
7,Diesel A1,PLANTA PUREO,Calbuco,Pureo,BO
8,Diesel A1,ENAP QUINTERO,NaN,Quintero,BU
9,Diesel A1,BT PUMA,NaN,Puma,CA
10,Gas 93,PLANTA MEJILLONES,Mejillones,Mejillones,DA


### Paso 6:

Extraemos la planificación y sus respectivas descargas. Se ignoran descargas realizadas por el Puma, Enap y por puertos hacia el Puma.

In [146]:
FIRST_ROW = 12
LAST_ROW = 61

def extraer_planificacion(file, sheet):
    df_planificacion = pd.read_excel(file, sheet_name=sheet, header=None)
    df_planificacion.index = range(1, len(df_planificacion) + 1)
    df_planificacion.columns = [int_to_excel_col(i) for i in range(1, len(df_planificacion.columns) + 1)]

    return df_planificacion

def extraer_descargas(df_planificacion, ignore_not_bts=False, df_bts=None):
    df_descargas = pd.DataFrame({"Fecha": [], "Abrev.": [], "Volumen": [], "Columna": []})
    columnas = ["M", "S", "Y", "AE", "AK", "AQ", "AW", "BC", "BI", "BO", "BU", "CA", "CF", "CK", "CP", "CU", "DA"]
    for col in columnas:
        descargas_parciales = df_planificacion.loc[FIRST_ROW:LAST_ROW, ["B", col, next_excel_col(col)]].dropna()
        descargas_parciales.columns = ["Fecha", "Abrev.", "Volumen"]
        descargas_parciales["Columna"] = [col] * len(descargas_parciales)
        df_descargas = pd.concat([df_descargas, descargas_parciales], ignore_index=True)
    if ignore_not_bts:
        df_descargas = df_descargas[df_descargas["Abrev."].isin(df_bts["Abrev."].tolist())]
    return df_descargas

In [147]:
df_planificacion = extraer_planificacion(PATH_PROGRAMACION, "Planificación")
df_descargas = extraer_descargas(df_planificacion, ignore_not_bts=True, df_bts=df_bts)
df_descargas.index = range(1, len(df_descargas) + 1)
df_descargas

c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\jaubele.ap\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Fecha,Abrev.,Volumen,Columna
1,2025-09-25 00:00:00,FE,4000,M
2,2025-09-26 00:00:00,FE,4000,M
3,2025-09-30 00:00:00,SM,6000,M
4,2025-10-01 00:00:00,SM,6000,M
5,2025-10-14 00:00:00,PK,6000,M
...,...,...,...,...
71,2025-10-14 00:00:00,AS,9000,BI
72,2025-10-15 00:00:00,AS,9000,BI
73,2025-10-27 00:00:00,SS,7500,BI
74,2025-10-28 00:00:00,SS,7500,BI


### Paso 7:

Mediante la el atributo "Columna" asociamos cada descarga a un par producto-planta.

In [148]:
df_descargas_productos_plantas = df_descargas.merge(df_productos_plantas, on=["Columna"]).drop(columns=["Columna"])
df_descargas_productos_plantas.index = range(1, len(df_descargas_productos_plantas) + 1)
df_descargas_productos_plantas

,Fecha,Abrev.,Volumen,Producto,Planta,Ciudad,Alias
1,2025-09-25 00:00:00,FE,4000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
2,2025-09-26 00:00:00,FE,4000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
3,2025-09-30 00:00:00,SM,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
4,2025-10-01 00:00:00,SM,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
5,2025-10-14 00:00:00,PK,6000,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique
...,...,...,...,...,...,...,...
71,2025-10-14 00:00:00,AS,9000,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel
72,2025-10-15 00:00:00,AS,9000,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel
73,2025-10-27 00:00:00,SS,7500,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel
74,2025-10-28 00:00:00,SS,7500,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel


### Paso 8:

Mediante la abreviatura asociamos cada descarga a un programa (Por esto es importante que la abreviatura sea única, o en su defecto utilizar otro atributo de carater único como correlativo o nombre de programa).
Se eliminarán todas las descargas que no tengan la abreviatura de un buque asociada (Ej: Caldera o TPI hacia el Puma).

In [149]:
df_descargas_completo = df_descargas_productos_plantas.merge(df_bts, on=["Abrev."]).drop(columns=["Abrev."])
df_descargas_completo = df_descargas_completo[["Fecha", "N° Referencia", "Nombre programa", "Nombre del BT",
                                               "Producto", "Planta", "Ciudad", "Alias", "Volumen"]]
df_descargas_completo.index = range(1, len(df_descargas_completo) + 1)
df_descargas_completo

,Fecha,N° Referencia,Nombre programa,Nombre del BT,Producto,Planta,Ciudad,Alias,Volumen
1,2025-09-25 00:00:00,CC 75/25,75/25 Flora Express,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,4000
2,2025-09-26 00:00:00,CC 75/25,75/25 Flora Express,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,4000
3,2025-09-30 00:00:00,CC 48/25,48/25 STI Mighty,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,6000
4,2025-10-01 00:00:00,CC 48/25,48/25 STI Mighty,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,6000
5,2025-10-14 00:00:00,CC 50/25,50/25 Pacific Kohinoor,Pacific Kohinoor,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,6000
...,...,...,...,...,...,...,...,...,...
71,2025-10-14 00:00:00,CC 118/25,118/25 Andean Sun,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,9000
72,2025-10-15 00:00:00,CC 118/25,118/25 Andean Sun,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,9000
73,2025-10-27 00:00:00,CC 130/25,130/25 Seaways Stamford,Seaways Stamford,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,7500
74,2025-10-28 00:00:00,CC 130/25,130/25 Seaways Stamford,Seaways Stamford,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,7500


### Paso 9:

Agrupamos todas las descargas de un programa que son en días consecutivos de un mismo producto y en una misma planta como una sola descarga.

Podemos notar que Seaways Kenosha y Seaways Kolberg quedan asociados a la misma descarga (cuando realmente solo corresponde al Kolberg).

In [150]:
def agrupar_descargas(df_descargas_completo):
    df = df_descargas_completo.copy()
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df["diff"] = (df.groupby(["Nombre programa", "Producto", "Planta"])["Fecha"].diff().dt.days)
    df["Grupo"] = (df["diff"] != 1).cumsum()
    df_descargas_agrupadas = (df.groupby("Grupo", as_index=False).agg({"Fecha": ["min", "max"],
                                                                       "Volumen": "sum"}))
    df_descargas_agrupadas.columns = ["Grupo", "Fecha inicio", "Fecha fin", "Volumen total"]
    df_descargas_completo_sin_duplicados = df.drop_duplicates(subset=["Grupo"])
    df_descargas_agrupadas = df_descargas_agrupadas.merge(
        df_descargas_completo_sin_duplicados, on="Grupo").drop(columns=["diff", "Fecha", "Volumen"])
    df_descargas_agrupadas = df_descargas_agrupadas[[
        "Nombre programa", "N° Referencia", "Nombre del BT", "Producto", "Planta", "Ciudad", "Alias",
        "Fecha inicio", "Fecha fin", "Volumen total"]]

    # Ordenar por Nombre programa y Fecha inicio, y agregar N° Descarga
    df_descargas_agrupadas = df_descargas_agrupadas.sort_values(by=["Nombre programa", "Fecha inicio"])
    df_descargas_agrupadas["N° Descarga"] = (df_descargas_agrupadas.groupby("Nombre programa").cumcount() + 1)
    df_descargas_agrupadas.index = range(1, len(df_descargas_agrupadas) + 1) 
    return df_descargas_agrupadas

In [151]:
df_descargas_agrupadas = agrupar_descargas(df_descargas_completo)
df_descargas_agrupadas

,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga
1,102/25 Pintail Pacific,CC 102/25,Pintail Pacific,Gas 93,BT PUMA,NaN,Puma,2025-09-23,2025-09-23,15260,1
2,102/25 Pintail Pacific,CC 102/25,Pintail Pacific,Gas 93,TERMINAL TPI,Quintero,TPI,2025-09-25,2025-09-25,34000,2
3,103/25 Seaways Wheat /High Challenge,CC 103/25,Seaways Wheat /High Challenge,Gas 97,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-23,2025-10-24,8000,1
4,103/25 Seaways Wheat /High Challenge,CC 103/25,Seaways Wheat /High Challenge,Gas 93,TERMINAL TPI,Quintero,TPI,2025-10-28,2025-10-29,40000,2
5,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09,2025-10-10,30000,1
6,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14,2025-10-15,18000,2
7,119/25 Seaways Dwarka,CC 119/25,Seaways Dwarka,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-10-23,2025-10-24,12000,1
8,119/25 Seaways Dwarka,CC 119/25,Seaways Dwarka,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-28,2025-10-29,36000,2
9,130/25 Seaways Stamford,CC 130/25,Seaways Stamford,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-23,2025-10-24,32000,1
10,130/25 Seaways Stamford,CC 130/25,Seaways Stamford,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-27,2025-10-28,15000,2


### Paso 10:

Extraemos todos los programas de la planificación.

In [152]:
def extraer_programas(df_planificacion):
    df_programas = df_planificacion.loc[FIRST_ROW:LAST_ROW, ["B", "J"]].dropna()
    df_programas.columns = ["ETA", "Nombre programa"]
    df_programas["ETA"] = pd.to_datetime(df_programas["ETA"]).apply(lambda dt: dt.replace(hour=0, minute=0, second=0) if pd.notna(dt) else dt)

    return df_programas

In [153]:
df_programas = extraer_programas(df_planificacion)
df_programas.index = range(1, len(df_programas) + 1)
df_programas

,ETA,Nombre programa
1,2025-09-25,75/25 Flora Express
2,2025-09-29,48/25 STI Mighty
3,2025-10-01,21/25 Seaways Athens
4,2025-10-06,118/25 Andean Sun
5,2025-10-07,49/25 Pacific Sentinel
6,2025-10-10,90/25 Dat Mercury
7,2025-10-13,50/25 Pacific Kohinoor
8,2025-10-14,20/25 Seaways Kolberg
9,2025-10-15,74/25 Gem Sapphire
10,2025-10-16,130/25 Seaways Stamford


### Paso 11:

Asociamos a cada programa sus respectivas descargas, agregamos la ETA a la primera descarga de cada programa.

* Nota Importante: Esto ignorará todas las descargas asociadas a programas cuya ETA estimada sea anterior al inicio de la programación.

In [154]:
df_descargas_agrupadas = df_programas.merge(df_descargas_agrupadas, on="Nombre programa")
df_descargas_agrupadas["ETA"] = df_descargas_agrupadas["ETA"][[True if descarga == 1 else False for descarga in df_descargas_agrupadas["N° Descarga"]]]
df_descargas_agrupadas.index = range(1, len(df_descargas_agrupadas) + 1)
df_descargas_agrupadas

,ETA,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga
1,2025-09-25,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25,2025-09-26,8000,1
2,NaT,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03,2025-10-04,42398,2
3,2025-09-29,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30,2025-10-01,12000,1
4,NaT,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04,2025-10-05,36000,2
5,2025-10-01,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04,2025-10-05,15000,1
6,NaT,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07,2025-10-08,33000,2
7,2025-10-06,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09,2025-10-10,30000,1
8,NaT,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14,2025-10-15,18000,2
9,2025-10-07,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09,2025-10-10,12000,1
10,NaT,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14,2025-10-15,36000,2


### Paso 12:

Extraer matriz de distancias y pasar a tiempos aproximando en horas al entero superior

In [155]:
def extraer_tiempos_de_viaje(file, sheet):
    VELOCIDAD_MEDIA = 12  # nudos
    df_distancias = pd.read_excel(file, sheet_name=sheet, header=3, index_col=0)
    df_distancias.dropna(axis=1, how='all', inplace=True)
    df_distancias.dropna(axis=0, how='all', inplace=True)
    matriz_de_tiempos = np.ceil(df_distancias / VELOCIDAD_MEDIA).astype(int)
    return matriz_de_tiempos

In [156]:
matriz_de_tiempos = extraer_tiempos_de_viaje(PATH_DISTANCIAS, "Datos")
matriz_de_tiempos

,ARICA,IQUIQUE,TOCOPILLA,MEJILLONES,ANTOFAGASTA,BARQUITO,CALDERA,HUASCO,GUAYACAN,QUINTERO,SAN VICENTE,CORONEL,PUERTO MONTT,CALBUCO,CHACABUCO,PUNTA ARENAS,CABO NEGRO
Arica,0,9,19,24,28,40,44,51,59,73,93,95,124,122,141,192,193
Iquique,9,0,10,15,19,32,35,43,51,64,85,87,116,113,133,183,185
Tocopilla,19,10,0,6,10,23,26,33,41,55,75,77,107,104,123,174,175
Mejillones,24,15,6,0,6,19,22,30,38,51,72,74,102,100,119,170,172
Antofagasta,28,19,10,6,0,15,18,26,34,48,68,70,99,97,116,166,168
Barquito,40,32,23,19,15,0,5,12,20,33,54,56,86,83,102,153,154
Caldera,44,35,26,22,18,5,0,8,17,31,51,53,82,80,99,149,151
Huasco,51,43,33,30,26,12,8,0,9,23,44,46,75,72,92,142,143
Guayacan,59,51,41,38,34,20,17,9,0,16,36,38,67,65,84,135,136
Quintero,73,64,55,51,48,33,31,23,16,0,22,24,54,51,70,121,122


### Paso 13:

Calcular ETAs según matriz de tiempos

In [157]:
def rellenar_etas(df_descargas_agrupadas, matriz_de_tiempos):
    for programa, group in df_descargas_agrupadas.groupby('Nombre programa'):
        group = group.sort_values('N° Descarga').copy()
        for i in range(len(group)):
            if pd.isna(group.iloc[i]['ETA']):
                if i == 0:
                    continue  # No se puede calcular ETA para la primera descarga sin ETA
                prev = group.iloc[i-1]
                ciudad_origen = prev['Ciudad']
                ciudad_destino = group.iloc[i]['Ciudad']

                # Hora de salida: fecha fin anterior a las 23:00
                hora_salida = prev['Fecha fin'].replace(hour=23, minute=0, second=0)
                
                # Si alguna ciudad es NaN (probablemente PUMA o ENAP), tiempo de viaje 0.
                if pd.isna(ciudad_origen) or pd.isna(ciudad_destino):
                    df_descargas_agrupadas.loc[group.index[i], 'ETA'] = hora_salida
                    continue
                horas_viaje = matriz_de_tiempos.loc[ciudad_origen, ciudad_destino.upper()]
                df_descargas_agrupadas.loc[group.index[i], 'ETA'] = hora_salida + timedelta(hours=int(horas_viaje))

In [158]:
rellenar_etas(df_descargas_agrupadas, matriz_de_tiempos)
df_descargas_agrupadas

,ETA,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga
1,2025-09-25 00:00:00,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25,2025-09-26,8000,1
2,2025-09-29 15:00:00,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03,2025-10-04,42398,2
3,2025-09-29 00:00:00,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30,2025-10-01,12000,1
4,2025-10-02 14:00:00,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04,2025-10-05,36000,2
5,2025-10-01 00:00:00,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04,2025-10-05,15000,1
6,2025-10-05 23:00:00,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07,2025-10-08,33000,2
7,2025-10-06 00:00:00,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09,2025-10-10,30000,1
8,2025-10-14 01:00:00,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14,2025-10-15,18000,2
9,2025-10-07 00:00:00,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09,2025-10-10,12000,1
10,2025-10-11 21:00:00,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14,2025-10-15,36000,2


### Paso 14:

Ajustamos al formato de la Base de Datos.

In [159]:
df_BD = df_descargas_agrupadas[["N° Referencia", "Nombre del BT", "Producto", "Alias", "Volumen total", "ETA", "Fecha fin"]]
df_BD.columns = ["CC", "Nombre BT", "Producto", "Puerto", "Volumen", "ETA", "Fin descarga"]
df_BD

,CC,Nombre BT,Producto,Puerto,Volumen,ETA,Fin descarga
1,CC 75/25,Flora Express,Diesel A1,Iquique,8000,2025-09-25 00:00:00,2025-09-26
2,CC 75/25,Flora Express,Diesel A1,TPI,42398,2025-09-29 15:00:00,2025-10-04
3,CC 48/25,STI Mighty,Diesel A1,Iquique,12000,2025-09-29 00:00:00,2025-10-01
4,CC 48/25,STI Mighty,Diesel A1,Mejillones,36000,2025-10-02 14:00:00,2025-10-05
5,CC 21/25,Seaways Athens,Diesel A1,Oxiquim Quintero,15000,2025-10-01 00:00:00,2025-10-05
6,CC 21/25,Seaways Athens,Diesel A1,TPI,33000,2025-10-05 23:00:00,2025-10-08
7,CC 118/25,Andean Sun,Diesel A1,Mejillones,30000,2025-10-06 00:00:00,2025-10-10
8,CC 118/25,Andean Sun,Diesel A1,Coronel,18000,2025-10-14 01:00:00,2025-10-15
9,CC 49/25,Pacific Sentinel,Diesel A1,Caldera,12000,2025-10-07 00:00:00,2025-10-10
10,CC 49/25,Pacific Sentinel,Diesel A1,Mejillones,36000,2025-10-11 21:00:00,2025-10-15


## Parte 2:

Agregar campos de Proveedor, Inicio Ventana, Final Ventana.

In [160]:
PATH_NUEVA_FICHA = "Nueva Ficha Información de Buques.xlsx"

In [161]:
def extraer_nueva_ficha(file, sheet):
    df_nueva_ficha = pd.read_excel(file, sheet_name=sheet, header=3)
    df_nueva_ficha = df_nueva_ficha[["N° Referencia", "Proveedor", "Inicio Ventana",
                                     "Fin Ventana", "Inicio Ventana Corta", "Fin Ventana Corta",
                                     "MONTO ($/DIA)"]]
    df_nueva_ficha.drop_duplicates(subset=["N° Referencia"], keep="first", inplace=True)
    
    return df_nueva_ficha

In [162]:
df_nueva_ficha = extraer_nueva_ficha(PATH_NUEVA_FICHA, "Programación de buques")
df_nueva_ficha = df_nueva_ficha[df_nueva_ficha["N° Referencia"].isin(df_BD["CC"])]
df_nueva_ficha.index = range(1, len(df_nueva_ficha) + 1)
df_nueva_ficha

,N° Referencia,Proveedor,Inicio Ventana,Fin Ventana,Inicio Ventana Corta,Fin Ventana Corta,MONTO ($/DIA)
1,CC 20/25,ARAMCO,2025-09-22,2025-09-26 00:00:00,2025-09-24 00:00:00,2025-09-26 00:00:00,34500
2,CC 75/25,PHILLIPS 66,2025-09-22,2025-09-26 00:00:00,2025-09-24 00:00:00,2025-09-26 00:00:00,32000
3,CC 21/25,ARAMCO,2025-09-29,2025-10-03 00:00:00,2025-10-01 00:00:00,2025-10-03 00:00:00,34000
4,CC 48/25,CHEVRON,2025-09-29,2025-10-03 00:00:00,2025-09-29 00:00:00,2025-10-01 00:00:00,40000
5,CC 49/25,CHEVRON,2025-10-06,2025-10-10 00:00:00,Pendiente,Pendiente,40000
6,CC 118/25,ARAMCO,2025-10-06,2025-10-10 00:00:00,Pendiente,Pendiente,33000
7,CC 90/25,CHEVRON,2025-10-08,2025-10-12 00:00:00,Pendiente,Pendiente,40000
8,CC 50/25,CHEVRON,2025-10-13,2025-10-17 00:00:00,Pendiente,Pendiente,33000
9,CC 74/25,MARATHON,2025-10-13,2025-10-17 00:00:00,Pendiente,Pendiente,NaN
10,CC 130/25,ARAMCO,2025-10-13,2025-10-17 00:00:00,Pendiente,Pendiente,35000


Establecemos la ventana corta como la ventana e caso de existir, en otro caso se conserva la ventana original.

In [163]:
for col in ["Inicio Ventana Corta", "Fin Ventana Corta"]:
    df_nueva_ficha[col] = pd.to_datetime(df_nueva_ficha[col], errors="coerce")

df_nueva_ficha["Inicio Ventana"] = df_nueva_ficha["Inicio Ventana Corta"].combine_first(pd.to_datetime(df_nueva_ficha["Inicio Ventana"]))
df_nueva_ficha["Final Ventana"] = df_nueva_ficha["Fin Ventana Corta"].combine_first(pd.to_datetime(df_nueva_ficha["Fin Ventana"]))
df_nueva_ficha.drop(columns=["Fin Ventana"], inplace=True)

# Eliminar columnas de ventanas cortas
df_nueva_ficha = df_nueva_ficha.drop(columns=["Inicio Ventana Corta", "Fin Ventana Corta"])
df_nueva_ficha

,N° Referencia,Proveedor,Inicio Ventana,MONTO ($/DIA),Final Ventana
1,CC 20/25,ARAMCO,2025-09-24,34500,2025-09-26
2,CC 75/25,PHILLIPS 66,2025-09-24,32000,2025-09-26
3,CC 21/25,ARAMCO,2025-10-01,34000,2025-10-03
4,CC 48/25,CHEVRON,2025-09-29,40000,2025-10-01
5,CC 49/25,CHEVRON,2025-10-06,40000,2025-10-10
6,CC 118/25,ARAMCO,2025-10-06,33000,2025-10-10
7,CC 90/25,CHEVRON,2025-10-08,40000,2025-10-12
8,CC 50/25,CHEVRON,2025-10-13,33000,2025-10-17
9,CC 74/25,MARATHON,2025-10-13,NaN,2025-10-17
10,CC 130/25,ARAMCO,2025-10-13,35000,2025-10-17


In [164]:
df_BD = df_BD.merge(df_nueva_ficha, left_on="CC", right_on="N° Referencia").drop(columns=["N° Referencia"])
df_BD = df_BD[["CC", "Nombre BT", "Proveedor", "Producto", "Puerto", "Volumen", "Inicio Ventana", "Final Ventana", "ETA", "Fin descarga"]]
df_BD.index = range(1, len(df_BD) + 1)
df_BD

,CC,Nombre BT,Proveedor,Producto,Puerto,Volumen,Inicio Ventana,Final Ventana,ETA,Fin descarga
1,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,Iquique,8000,2025-09-24,2025-09-26,2025-09-25 00:00:00,2025-09-26
2,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,TPI,42398,2025-09-24,2025-09-26,2025-09-29 15:00:00,2025-10-04
3,CC 48/25,STI Mighty,CHEVRON,Diesel A1,Iquique,12000,2025-09-29,2025-10-01,2025-09-29 00:00:00,2025-10-01
4,CC 48/25,STI Mighty,CHEVRON,Diesel A1,Mejillones,36000,2025-09-29,2025-10-01,2025-10-02 14:00:00,2025-10-05
5,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,Oxiquim Quintero,15000,2025-10-01,2025-10-03,2025-10-01 00:00:00,2025-10-05
6,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,TPI,33000,2025-10-01,2025-10-03,2025-10-05 23:00:00,2025-10-08
7,CC 118/25,Andean Sun,ARAMCO,Diesel A1,Mejillones,30000,2025-10-06,2025-10-10,2025-10-06 00:00:00,2025-10-10
8,CC 118/25,Andean Sun,ARAMCO,Diesel A1,Coronel,18000,2025-10-06,2025-10-10,2025-10-14 01:00:00,2025-10-15
9,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,Caldera,12000,2025-10-06,2025-10-10,2025-10-07 00:00:00,2025-10-10
10,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,Mejillones,36000,2025-10-06,2025-10-10,2025-10-11 21:00:00,2025-10-15


## Parte 3:

Se agrega fecha de programación y se calcula la semana de manera automática. En la app se selecciona a través de un form.

In [165]:
def get_week_of_month(year, month, day):
    month_calendar = calendar.monthcalendar(year, month)
    for week_number, week in enumerate(month_calendar, start=1):
        if day in week:
            return "Semana " + str(week_number)

In [166]:
FECHA_PROGRAMACION = pd.to_datetime("2025-09-24")  # Ejemplo, en la app se selecciona a través de un form
columnas_vacias = ["Fecha de programación", "Semana", "Año", "Mes", "Horas Laytime", 
                "Demurrage"]
for col in columnas_vacias:
    df_BD[col] = [np.nan] * len(df_BD)

df_BD["Fecha de programación"] = FECHA_PROGRAMACION
df_BD["Semana"] = get_week_of_month(FECHA_PROGRAMACION.year, FECHA_PROGRAMACION.month, FECHA_PROGRAMACION.day)

# Reordenar columnas
df_BD = df_BD[["Fecha de programación", "Semana", "Año", "Mes", "Horas Laytime",
            "CC", "Nombre BT", "Proveedor", "Producto", "Demurrage", "Puerto",
            "Volumen", "Inicio Ventana", "Final Ventana", "ETA", "Fin descarga"]]
df_BD


,Fecha de programación,Semana,Año,Mes,Horas Laytime,CC,Nombre BT,Proveedor,Producto,Demurrage,Puerto,Volumen,Inicio Ventana,Final Ventana,ETA,Fin descarga
1,2025-09-24,Semana 4,NaN,NaN,NaN,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,NaN,Iquique,8000,2025-09-24,2025-09-26,2025-09-25 00:00:00,2025-09-26
2,2025-09-24,Semana 4,NaN,NaN,NaN,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,NaN,TPI,42398,2025-09-24,2025-09-26,2025-09-29 15:00:00,2025-10-04
3,2025-09-24,Semana 4,NaN,NaN,NaN,CC 48/25,STI Mighty,CHEVRON,Diesel A1,NaN,Iquique,12000,2025-09-29,2025-10-01,2025-09-29 00:00:00,2025-10-01
4,2025-09-24,Semana 4,NaN,NaN,NaN,CC 48/25,STI Mighty,CHEVRON,Diesel A1,NaN,Mejillones,36000,2025-09-29,2025-10-01,2025-10-02 14:00:00,2025-10-05
5,2025-09-24,Semana 4,NaN,NaN,NaN,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,NaN,Oxiquim Quintero,15000,2025-10-01,2025-10-03,2025-10-01 00:00:00,2025-10-05
6,2025-09-24,Semana 4,NaN,NaN,NaN,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,NaN,TPI,33000,2025-10-01,2025-10-03,2025-10-05 23:00:00,2025-10-08
7,2025-09-24,Semana 4,NaN,NaN,NaN,CC 118/25,Andean Sun,ARAMCO,Diesel A1,NaN,Mejillones,30000,2025-10-06,2025-10-10,2025-10-06 00:00:00,2025-10-10
8,2025-09-24,Semana 4,NaN,NaN,NaN,CC 118/25,Andean Sun,ARAMCO,Diesel A1,NaN,Coronel,18000,2025-10-06,2025-10-10,2025-10-14 01:00:00,2025-10-15
9,2025-09-24,Semana 4,NaN,NaN,NaN,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,NaN,Caldera,12000,2025-10-06,2025-10-10,2025-10-07 00:00:00,2025-10-10
10,2025-09-24,Semana 4,NaN,NaN,NaN,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,NaN,Mejillones,36000,2025-10-06,2025-10-10,2025-10-11 21:00:00,2025-10-15


## Parte 4:

Agsignar mes y año a programa en base al mes donde efectúe mayor volumen de descarga. Ejemplo CC 47/25 STI Mighty tiene 6.000 de descarga en septiembre y 42.000 en octubre se asigna completo a octubre.

In [167]:
MESES = {1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 
         5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto", 
         9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"}

In [168]:
def asignar_año_mes(df_descargas_completo, df_BD):
    df = df_descargas_completo.copy()
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df["Mes_Año"] = df["Fecha"].dt.to_period("M")
    vol_por_mes = df.groupby(["N° Referencia", "Mes_Año"], as_index=False)["Volumen"].sum()
    mes_mayor_volumen = vol_por_mes.loc[vol_por_mes.groupby("N° Referencia")["Volumen"].idxmax()]
    df_BD = df_BD.merge(mes_mayor_volumen[["N° Referencia", "Mes_Año"]],
                left_on="CC", right_on="N° Referencia", how="left").drop(columns=["N° Referencia"])
    df_BD["Mes"] = df_BD["Mes_Año"].dt.month
    df_BD["Año"] = df_BD["Mes_Año"].dt.year
    df_BD.drop(columns=["Mes_Año"], inplace=True)
    df_BD.index = range(1, len(df_BD) + 1)
    df_BD["Mes"] = df_BD["Mes"].map(MESES)
    
    return df_BD

In [169]:
df_BD = asignar_año_mes(df_descargas_completo, df_BD)
df_BD

,Fecha de programación,Semana,Año,Mes,Horas Laytime,CC,Nombre BT,Proveedor,Producto,Demurrage,Puerto,Volumen,Inicio Ventana,Final Ventana,ETA,Fin descarga
1,2025-09-24,Semana 4,2025,Octubre,NaN,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,NaN,Iquique,8000,2025-09-24,2025-09-26,2025-09-25 00:00:00,2025-09-26
2,2025-09-24,Semana 4,2025,Octubre,NaN,CC 75/25,Flora Express,PHILLIPS 66,Diesel A1,NaN,TPI,42398,2025-09-24,2025-09-26,2025-09-29 15:00:00,2025-10-04
3,2025-09-24,Semana 4,2025,Octubre,NaN,CC 48/25,STI Mighty,CHEVRON,Diesel A1,NaN,Iquique,12000,2025-09-29,2025-10-01,2025-09-29 00:00:00,2025-10-01
4,2025-09-24,Semana 4,2025,Octubre,NaN,CC 48/25,STI Mighty,CHEVRON,Diesel A1,NaN,Mejillones,36000,2025-09-29,2025-10-01,2025-10-02 14:00:00,2025-10-05
5,2025-09-24,Semana 4,2025,Octubre,NaN,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,NaN,Oxiquim Quintero,15000,2025-10-01,2025-10-03,2025-10-01 00:00:00,2025-10-05
6,2025-09-24,Semana 4,2025,Octubre,NaN,CC 21/25,Seaways Athens,ARAMCO,Diesel A1,NaN,TPI,33000,2025-10-01,2025-10-03,2025-10-05 23:00:00,2025-10-08
7,2025-09-24,Semana 4,2025,Octubre,NaN,CC 118/25,Andean Sun,ARAMCO,Diesel A1,NaN,Mejillones,30000,2025-10-06,2025-10-10,2025-10-06 00:00:00,2025-10-10
8,2025-09-24,Semana 4,2025,Octubre,NaN,CC 118/25,Andean Sun,ARAMCO,Diesel A1,NaN,Coronel,18000,2025-10-06,2025-10-10,2025-10-14 01:00:00,2025-10-15
9,2025-09-24,Semana 4,2025,Octubre,NaN,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,NaN,Caldera,12000,2025-10-06,2025-10-10,2025-10-07 00:00:00,2025-10-10
10,2025-09-24,Semana 4,2025,Octubre,NaN,CC 49/25,Pacific Sentinel,CHEVRON,Diesel A1,NaN,Mejillones,36000,2025-10-06,2025-10-10,2025-10-11 21:00:00,2025-10-15


## Parte 5: Cálculo Laytime/Demurrage

In [170]:
df_descargas_agrupadas_con_ventanas = df_descargas_agrupadas.merge(
    df_BD[["CC", "ETA", "Inicio Ventana", "Final Ventana"]],
    left_on=["N° Referencia", "ETA"], right_on=["CC", "ETA"], how="left").drop(columns=["CC"])

df_descargas_agrupadas_con_ventanas["Inicio Ventana"] = pd.to_datetime(df_descargas_agrupadas_con_ventanas["Inicio Ventana"]).apply(lambda dt: dt.replace(hour=0, minute=0, second=0) if pd.notna(dt) else dt)
df_descargas_agrupadas_con_ventanas["Final Ventana"] = pd.to_datetime(df_descargas_agrupadas_con_ventanas["Final Ventana"]).apply(lambda dt: dt.replace(hour=23, minute=59, second=59) if pd.notna(dt) else dt)
df_descargas_agrupadas_con_ventanas["Fecha inicio"] = pd.to_datetime(df_descargas_agrupadas_con_ventanas["Fecha inicio"]).apply(lambda dt: dt.replace(hour=15, minute=0, second=0) if pd.notna(dt) else dt)
df_descargas_agrupadas_con_ventanas["Fecha fin"] = pd.to_datetime(df_descargas_agrupadas_con_ventanas["Fecha fin"]).apply(lambda dt: dt.replace(hour=23, minute=0, second=0) if pd.notna(dt) else dt)

# ETA en ventana o num de descarga > 1
inicio_laytime = pd.concat([
    df_descargas_agrupadas_con_ventanas['Fecha inicio'],
    df_descargas_agrupadas_con_ventanas['ETA'] + pd.Timedelta(hours=6)
], axis=1).min(axis=1)

# Condición N° de descarga = 1 y ETA < Inicio Ventana
cond = (df_descargas_agrupadas_con_ventanas['N° Descarga'] == 1) & (df_descargas_agrupadas_con_ventanas['ETA'] < df_descargas_agrupadas_con_ventanas['Inicio Ventana'])
inicio_laytime = inicio_laytime.mask(cond, pd.concat([df_descargas_agrupadas_con_ventanas['Inicio Ventana'], df_descargas_agrupadas_con_ventanas['Fecha inicio']], axis=1).min(axis=1))

# Condición N° de descarga = 1 y ETA > Final Ventana
cond = (df_descargas_agrupadas_con_ventanas['N° Descarga'] == 1) & (df_descargas_agrupadas_con_ventanas['ETA'] > df_descargas_agrupadas_con_ventanas['Final Ventana'])
inicio_laytime = inicio_laytime.mask(cond, df_descargas_agrupadas_con_ventanas['Fecha inicio'])

# Asignamos la columna final
df_descargas_agrupadas_con_ventanas['Inicio Laytime'] = inicio_laytime
df_descargas_agrupadas_con_ventanas["Laytime"] = (df_descargas_agrupadas_con_ventanas["Fecha fin"] - df_descargas_agrupadas_con_ventanas["Inicio Laytime"]).dt.total_seconds() / 3600
df_descargas_agrupadas_con_ventanas

,ETA,Nombre programa,N° Referencia,Nombre del BT,Producto,Planta,Ciudad,Alias,Fecha inicio,Fecha fin,Volumen total,N° Descarga,Inicio Ventana,Final Ventana,Inicio Laytime,Laytime
0,2025-09-25 00:00:00,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-25 15:00:00,2025-09-26 23:00:00,8000,1,2025-09-24,2025-09-26 23:59:59,2025-09-25 06:00:00,41.0
1,2025-09-29 15:00:00,75/25 Flora Express,CC 75/25,Flora Express,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-03 15:00:00,2025-10-04 23:00:00,42398,2,2025-09-24,2025-09-26 23:59:59,2025-09-29 21:00:00,122.0
2,2025-09-29 00:00:00,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA IQUIQUE,Iquique,Iquique,2025-09-30 15:00:00,2025-10-01 23:00:00,12000,1,2025-09-29,2025-10-01 23:59:59,2025-09-29 06:00:00,65.0
3,2025-10-02 14:00:00,48/25 STI Mighty,CC 48/25,STI Mighty,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-04 15:00:00,2025-10-05 23:00:00,36000,2,2025-09-29,2025-10-01 23:59:59,2025-10-02 20:00:00,75.0
4,2025-10-01 00:00:00,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,OXIQUIM QUINTERO,Quintero,Oxiquim Quintero,2025-10-04 15:00:00,2025-10-05 23:00:00,15000,1,2025-10-01,2025-10-03 23:59:59,2025-10-01 06:00:00,113.0
5,2025-10-05 23:00:00,21/25 Seaways Athens,CC 21/25,Seaways Athens,Diesel A1,TERMINAL TPI,Quintero,TPI,2025-10-07 15:00:00,2025-10-08 23:00:00,33000,2,2025-10-01,2025-10-03 23:59:59,2025-10-06 05:00:00,66.0
6,2025-10-06 00:00:00,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-09 15:00:00,2025-10-10 23:00:00,30000,1,2025-10-06,2025-10-10 23:59:59,2025-10-06 06:00:00,113.0
7,2025-10-14 01:00:00,118/25 Andean Sun,CC 118/25,Andean Sun,Diesel A1,OXIQUIM CORONEL,Coronel,Coronel,2025-10-14 15:00:00,2025-10-15 23:00:00,18000,2,2025-10-06,2025-10-10 23:59:59,2025-10-14 07:00:00,40.0
8,2025-10-07 00:00:00,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA CALDERA,Caldera,Caldera,2025-10-09 15:00:00,2025-10-10 23:00:00,12000,1,2025-10-06,2025-10-10 23:59:59,2025-10-07 06:00:00,89.0
9,2025-10-11 21:00:00,49/25 Pacific Sentinel,CC 49/25,Pacific Sentinel,Diesel A1,PLANTA MEJILLONES,Mejillones,Mejillones,2025-10-14 15:00:00,2025-10-15 23:00:00,36000,2,2025-10-06,2025-10-10 23:59:59,2025-10-12 03:00:00,92.0
